In [92]:
import json
import pandas as pd
import re

In [93]:
from synthetic_invoice_generator import *

In [94]:
def json_to_df(filename):
    '''
    converts json string from json file to pd dataframe
    '''
    with open(filename) as file:
        data = json.load(file)

    formatted_data = []
    for item in data:
        row = {entry["field"]: entry["value"] for entry in item["extractions"]}
        formatted_data.append(row)

    df = pd.DataFrame(formatted_data)
    return df

In [95]:
def string_to_df(json_string):
    '''
    converts json string to pd dataframe
    '''
    formatted_data = []
    for item in json_string:
        row = {entry["field"]: entry["value"] for entry in item["extractions"]}
        formatted_data.append(row)

    df = pd.DataFrame(formatted_data)
    return df

In [99]:
data = generate_dataset(num_invoices=1000,num_merchants=10000,anomaly_rate=0.4, seed=42)
df = string_to_df(data)
df

Generating 1000 synthetic invoices across a set of 10000 merchants (anomaly rate: 40.0%)...
Analysis of 1000 invoices:
- Unique merchants: 915
- Total line items: 3440
- Average line items per invoice: 3.44

Field frequency:
- merchant: 1000 (100.0%)
- invoice_date: 1000 (100.0%)
- merchant_branch: 1000 (100.0%)
- merchant_chain: 1000 (100.0%)
- due_date: 1000 (100.0%)
- payment_terms: 1000 (100.0%)
- grand_total: 1000 (100.0%)
- tax: 1000 (100.0%)
- po_number: 1000 (100.0%)
- merchant_address: 1000 (100.0%)
- payment_method: 1000 (100.0%)
- country: 1000 (100.0%)
- currency: 1000 (100.0%)
- line_details: 1000 (100.0%)
Generated 1000 synthetic invoices and saved to synthetic_invoices.json
Saved invoices in JSONL format to synthetic_invoices.jsonl


,merchant,invoice_date,merchant_branch,merchant_chain,due_date,payment_terms,grand_total,tax,po_number,merchant_address,payment_method,country,currency,line_details
0,Barry,01/26/2025,Barry,Barry,01/26/2025,DUE ON RECEIPT,47066.95,3191.51,CUST-68053-2248,141 Hodges Mountains Kramerville NM 04216 USA,UPS Ground,US,USD,"[{'line_count': '1', 'line_description': 'LEU-..."
1,Macdonald-Patterson Office Supplies International,01/24/2025,Macdonald-Patterson Office Supplies International,Macdonald-Patterson Office Supplies International,01/24/2025,DUE ON RECEIPT,14132.06,1228.07,K97225,57081 Wilkinson Cove East David OR 03013 USA,FedEx Express,US,USD,"[{'line_count': '1', 'line_description': 'QAT-..."
2,Sawyer Laboratory,02/09/2025,Sawyer Laboratory,Sawyer Laboratory,04/10/2025,NET 60 DAYS,15116.34,689.25,2503-2474,193 Ryan Lodge Suite 052 West Sylviachester NC...,FedEx Ground,US,USD,"[{'line_count': '1', 'line_description': 'QSM-..."
3,Stark-Church Enterprises,02/16/2025,Stark-Church Enterprises,Stark-Church Enterprises,03/18/2025,NET 30 DAYS,143731.61,12442.89,J85368,413 James Roads Bakerville IA 12699 USA,FedEx Ground,US,USD,"[{'line_count': '1', 'line_description': 'D-04..."
4,Bond Corp.,12/10/2024,Bond Corp.,Bond Corp.,01/09/2025,NET 30 DAYS,7280.17,425.01,PO-16917,746 Michael Trafficway Michelleburgh NV 02384 USA,UPS Ground,US,USD,"[{'line_count': '1', 'line_description': 'YFR-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Williams & Jones,03/01/2025,Williams & Jones,Williams & Jones,03/31/2025,NET 30 DAYS,4903.70,316.88,2503-3866,57814 Allen Cape Port Katie NV 77018 USA,USPS Priority,US,USD,"[{'line_count': '1', 'line_description': 'MT-1..."
996,Smile GmbH,09/24/2024,Smile GmbH,Smile GmbH,11/23/2024,NET 60 DAYS,23873.80,779.79,L91957,178 Deanna Locks Matthewborough NM 08156 USA,ACH,US,USD,"[{'line_count': '1', 'line_description': 'WCW-..."
997,Jackson-Smith,01/15/2025,Jackson-Smith,Jackson-Smith,01/15/2025,DUE ON RECEIPT,33015.90,1511.58,CUST-74331-4375,365 Laura Grove Suite 597 West Jaimefort SC 63...,UPS Ground,US,USD,"[{'line_count': '1', 'line_description': 'D-04..."
998,Short-Fields Ltd.,02/18/2025,Short-Fields Ltd.,Short-Fields Ltd.,04/04/2025,NET 45 DAYS,2536.50,128.22,61659-302,8546 Catherine Flat Suite 401 North Dustinburg...,FedEx Express,US,USD,"[{'line_count': '1', 'line_description': 'KY-9..."


In [100]:
# Convert data types
df["grand_total"] = df["grand_total"].astype(float)
df["tax"] = df["tax"].astype(float)
df["due_date"] = pd.to_datetime(df["due_date"], errors="coerce")
df["invoice_date"] = pd.to_datetime(df["invoice_date"], errors="coerce")

# Derived features
df["invoice_age"] = (df["due_date"] - df["invoice_date"]).dt.days
df["num_line_items"] = df["line_details"].apply(lambda x: len(x) if isinstance(x, list) else 0)
df["avg_line_price"] = df["grand_total"] / df["num_line_items"].replace(0, np.nan)
df["avg_line_price"] = df["avg_line_price"].round(2)


df

,merchant,invoice_date,merchant_branch,merchant_chain,due_date,payment_terms,grand_total,tax,po_number,merchant_address,payment_method,country,currency,line_details,invoice_age,num_line_items,avg_line_price
0,Barry,2025-01-26,Barry,Barry,2025-01-26,DUE ON RECEIPT,47066.95,3191.51,CUST-68053-2248,141 Hodges Mountains Kramerville NM 04216 USA,UPS Ground,US,USD,"[{'line_count': '1', 'line_description': 'LEU-...",0,3,15688.98
1,Macdonald-Patterson Office Supplies International,2025-01-24,Macdonald-Patterson Office Supplies International,Macdonald-Patterson Office Supplies International,2025-01-24,DUE ON RECEIPT,14132.06,1228.07,K97225,57081 Wilkinson Cove East David OR 03013 USA,FedEx Express,US,USD,"[{'line_count': '1', 'line_description': 'QAT-...",0,4,3533.02
2,Sawyer Laboratory,2025-02-09,Sawyer Laboratory,Sawyer Laboratory,2025-04-10,NET 60 DAYS,15116.34,689.25,2503-2474,193 Ryan Lodge Suite 052 West Sylviachester NC...,FedEx Ground,US,USD,"[{'line_count': '1', 'line_description': 'QSM-...",60,1,15116.34
3,Stark-Church Enterprises,2025-02-16,Stark-Church Enterprises,Stark-Church Enterprises,2025-03-18,NET 30 DAYS,143731.61,12442.89,J85368,413 James Roads Bakerville IA 12699 USA,FedEx Ground,US,USD,"[{'line_count': '1', 'line_description': 'D-04...",30,4,35932.90
4,Bond Corp.,2024-12-10,Bond Corp.,Bond Corp.,2025-01-09,NET 30 DAYS,7280.17,425.01,PO-16917,746 Michael Trafficway Michelleburgh NV 02384 USA,UPS Ground,US,USD,"[{'line_count': '1', 'line_description': 'YFR-...",30,5,1456.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Williams & Jones,2025-03-01,Williams & Jones,Williams & Jones,2025-03-31,NET 30 DAYS,4903.70,316.88,2503-3866,57814 Allen Cape Port Katie NV 77018 USA,USPS Priority,US,USD,"[{'line_count': '1', 'line_description': 'MT-1...",30,4,1225.92
996,Smile GmbH,2024-09-24,Smile GmbH,Smile GmbH,2024-11-23,NET 60 DAYS,23873.80,779.79,L91957,178 Deanna Locks Matthewborough NM 08156 USA,ACH,US,USD,"[{'line_count': '1', 'line_description': 'WCW-...",60,3,7957.93
997,Jackson-Smith,2025-01-15,Jackson-Smith,Jackson-Smith,2025-01-15,DUE ON RECEIPT,33015.90,1511.58,CUST-74331-4375,365 Laura Grove Suite 597 West Jaimefort SC 63...,UPS Ground,US,USD,"[{'line_count': '1', 'line_description': 'D-04...",0,5,6603.18
998,Short-Fields Ltd.,2025-02-18,Short-Fields Ltd.,Short-Fields Ltd.,2025-04-04,NET 45 DAYS,2536.50,128.22,61659-302,8546 Catherine Flat Suite 401 North Dustinburg...,FedEx Express,US,USD,"[{'line_count': '1', 'line_description': 'KY-9...",45,3,845.50


In [104]:
with open("synthetic_data_configs.json", "r") as f:
    config = json.load(f)
state_tax_rates = config["tax_rates"]

# Function to extract state from address (format: "Street, City, State ZIP, Country")
def extract_state(address):
    match = re.search(r'\b[A-Z]{2}\b', address)  # Match two-letter state codes
    return match.group(0) if match else None

# Apply state extraction
df["state"] = df["merchant_address"].apply(extract_state)

# Map expected tax rate based on extracted state
df["expected_tax_rate"] = df["state"].map(state_tax_rates)

# State D.N.E if expected_tax_rate == NaN
df["fault_address_flag"] = df["expected_tax_rate"].isna().astype(int)

def compute_subtotal(line_details):
    return sum(float(line["line_total"]) for line in line_details)

df["sub_total"] = df["line_details"].apply(compute_subtotal)
df["cal_tax_rate"] = df["tax"] / df["sub_total"]
df["tax_mismatch_flag"] = (df["cal_tax_rate"].round(4) != df["expected_tax_rate"].round(4)).astype(int)

df

,merchant,invoice_date,merchant_branch,merchant_chain,due_date,payment_terms,grand_total,tax,po_number,merchant_address,...,line_details,invoice_age,num_line_items,avg_line_price,state,expected_tax_rate,fault_address_flag,sub_total,cal_tax_rate,tax_mismatch_flag
0,Barry,2025-01-26,Barry,Barry,2025-01-26,DUE ON RECEIPT,47066.95,3191.51,CUST-68053-2248,141 Hodges Mountains Kramerville NM 04216 USA,...,"[{'line_count': '1', 'line_description': 'LEU-...",0,3,15688.98,NM,0.0513,0,43840.91,0.072798,1
1,Macdonald-Patterson Office Supplies International,2025-01-24,Macdonald-Patterson Office Supplies International,Macdonald-Patterson Office Supplies International,2025-01-24,DUE ON RECEIPT,14132.06,1228.07,K97225,57081 Wilkinson Cove East David OR 03013 USA,...,"[{'line_count': '1', 'line_description': 'QAT-...",0,4,3533.02,OR,0.0000,0,12903.99,0.095170,1
2,Sawyer Laboratory,2025-02-09,Sawyer Laboratory,Sawyer Laboratory,2025-04-10,NET 60 DAYS,15116.34,689.25,2503-2474,193 Ryan Lodge Suite 052 West Sylviachester NC...,...,"[{'line_count': '1', 'line_description': 'QSM-...",60,1,15116.34,NC,0.0475,0,14396.07,0.047878,1
3,Stark-Church Enterprises,2025-02-16,Stark-Church Enterprises,Stark-Church Enterprises,2025-03-18,NET 30 DAYS,143731.61,12442.89,J85368,413 James Roads Bakerville IA 12699 USA,...,"[{'line_count': '1', 'line_description': 'D-04...",30,4,35932.90,IA,0.0600,0,131288.72,0.094775,1
4,Bond Corp.,2024-12-10,Bond Corp.,Bond Corp.,2025-01-09,NET 30 DAYS,7280.17,425.01,PO-16917,746 Michael Trafficway Michelleburgh NV 02384 USA,...,"[{'line_count': '1', 'line_description': 'YFR-...",30,5,1456.03,NV,0.0685,0,6855.16,0.061999,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Williams & Jones,2025-03-01,Williams & Jones,Williams & Jones,2025-03-31,NET 30 DAYS,4903.70,316.88,2503-3866,57814 Allen Cape Port Katie NV 77018 USA,...,"[{'line_count': '1', 'line_description': 'MT-1...",30,4,1225.92,NV,0.0685,0,4586.82,0.069085,1
996,Smile GmbH,2024-09-24,Smile GmbH,Smile GmbH,2024-11-23,NET 60 DAYS,23873.80,779.79,L91957,178 Deanna Locks Matthewborough NM 08156 USA,...,"[{'line_count': '1', 'line_description': 'WCW-...",60,3,7957.93,NM,0.0513,0,23094.01,0.033766,1
997,Jackson-Smith,2025-01-15,Jackson-Smith,Jackson-Smith,2025-01-15,DUE ON RECEIPT,33015.90,1511.58,CUST-74331-4375,365 Laura Grove Suite 597 West Jaimefort SC 63...,...,"[{'line_count': '1', 'line_description': 'D-04...",0,5,6603.18,SC,0.0600,0,31474.20,0.048026,1
998,Short-Fields Ltd.,2025-02-18,Short-Fields Ltd.,Short-Fields Ltd.,2025-04-04,NET 45 DAYS,2536.50,128.22,61659-302,8546 Catherine Flat Suite 401 North Dustinburg...,...,"[{'line_count': '1', 'line_description': 'KY-9...",45,3,845.50,TX,0.0625,0,2408.28,0.053241,1
